# Протоколы аутентификации

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

## Протокол PAP
![PAP Protocol](./images/PAP_1.png)

-----

In [3]:
# Использую хэш-функцию ГОСТ 34.11-2012 "Стрибог"
from pygost.gost34112012256 import GOST34112012256

# Эта либа тоже понадобится
import binascii

# Для генерации случайных чисел (КГПСЧ)
from Crypto.Random import get_random_bytes

----

In [4]:
def streebog_hash(*args) -> bytes:
    """
    Расчет хэша для нескольких аргументов (по типу конкатенации)
    """
    
    hasher = GOST34112012256()
    
    for arg in args:
        hasher.update(arg)
        
    return hasher.digest()

def bytes_as_hex(b: bytes) -> str:
    """
    Перевод бинарных данных в hex-строчку
    """
    
    return binascii.hexlify(b).decode()

------

In [5]:
class PAPServer(object):
    def __init__(self):
        """
        Инициализация сервера. 
        Создается пустая база с пользователями.
        """
        
        self._db = {}  # { login: hash(password) }
        
        
    def register_user(self, login: str, password: str):
        """
        Регистрация юзера. На вход получает логин и пароль.
        Ничего не возвращает.
        Выбрасывает исключение в случае наличия пользователя в базе
        """
        
        trace('[PAP Server]', f'Attempt to register user with credentials {login}:{password}')
        
        if login in self._db:
            trace('[PAP Server]', 'User already exists')
            raise ValueError(f'User {login} is already registered')
            
        #
        # Рассчитаю хэш пароля и запишу в базу
        #
        
        self._db[login] = streebog_hash(password.encode())
        trace('[PAP Server]', f'User successfully registered, password hash: {bytes_as_hex(self._db[login])}')
        
        
    def login(self, login: str, password: str) -> bool:
        """
        Вход пользователя по паролю. На вход получает логин и пароль.
        Возвращает True в случае успеха, False - иначе.
        """
        
        trace('[PAP Server]', f'Attempt to login with credentials {login}:{password}')
        
        if login not in self._db:
            trace('[PAP Server]', "User doesn't exist")
            return False
        
        #
        # Пользователь есть, рассчитаю хэш пароля и сравню с имеющимся
        #
        
        pass_hash = streebog_hash(password.encode())
        real_hash = self._db[login]
        
        trace('[PAP Server]', f'Password hash: {bytes_as_hex(pass_hash)}')
        trace('[PAP Server]', f'Stored hash:   {bytes_as_hex(real_hash)}')
        
        return pass_hash == real_hash

In [6]:
class PAPClient(object):
    def __init__(self, login, password):
        """
        Инициализация клиента. На вход получает логин и пароль.
        """
        
        self._login  = login
        self._passwd = password
        
        trace('[PAP Client]', f'User with credentials {self._login}:{self._passwd} created')
        
        
    def register(self, srv: PAPServer):
        """
        Регистрация пользователя на сервере. На вход принимает дескриптор сервера.
        """
        
        try:
            srv.register_user(self._login, self._passwd)
        except Exception as e:
            trace(e)
    
    
    def login(self, srv: PAPServer):
        """
        Вход пользователя на сервере. На вход принимает дескриптор сервера.
        """
        
        if srv.login(self._login, self._passwd):
            print('[PAP Client]', f'User {self._login} logged in successfully')
        else:
            print('[PAP Client]', f'Wrong username or password')

----

In [7]:
# Создаю сервер
server = PAPServer()

In [8]:
# Теперь создаю двух юзеров: 
# - Алису - валидного пользователя
# - Еву - пользователя, пытающегося представиться Алисой
alice = PAPClient('Alice', 'Trust_m3_0r_n0t')
eve = PAPClient('Alice', 'try_t0_gu3$$')

[TRACE] [PAP Client] User with credentials Alice:Trust_m3_0r_n0t created
[TRACE] [PAP Client] User with credentials Alice:try_t0_gu3$$ created


In [9]:
# Регистрирую валидного пользователя
alice.register(server)

[TRACE] [PAP Server] Attempt to register user with credentials Alice:Trust_m3_0r_n0t
[TRACE] [PAP Server] User successfully registered, password hash: bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8


In [10]:
# Захожу под кредами валидного пользователя
alice.login(server)

[TRACE] [PAP Server] Attempt to login with credentials Alice:Trust_m3_0r_n0t
[TRACE] [PAP Server] Password hash: bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8
[TRACE] [PAP Server] Stored hash:   bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8
[PAP Client] User Alice logged in successfully


In [11]:
# Пытаюсь зайти под неправильными кредами
eve.login(server)

[TRACE] [PAP Server] Attempt to login with credentials Alice:try_t0_gu3$$
[TRACE] [PAP Server] Password hash: 034eda071f984d17f28f9ae214c5d9b9a9fc0e112df98d0e4b5b2eb3d701b314
[TRACE] [PAP Server] Stored hash:   bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8
[PAP Client] Wrong username or password


-----

## Протокол CHAP
![CHAP Protocol](./images/CHAP_1.png)

In [12]:
class CHAPServer(object):
    def __init__(self):
        """
        Инициализация сервера. 
        Создается пустая база с пользователями.
        """
        
        self._db = {}
        
        
    def register_user(self, login, password):
        """
        Регистрация юзера. На вход получает логин и пароль.
        Ничего не возвращает.
        Выбрасывает исключение в случае наличия пользователя в базе
        """
        
        trace('[CHAP Server]', f'Attempt to register user with credentials {login}:{password}')
        
        if login in self._db:
            trace('[CHAP Server]', 'User already exists')
            raise ValueError(f'User {login} is already registered')
            
        #
        # Запишу в базу
        #
        
        self._db[login] = [password.encode(), None]
        trace('[CHAP Server]', f'User successfully registered')
    
    
    def login(self, login, response):
        """
        Вход пользователя. На вход получает логин и хэш пароля с челленджем.
        Возвращает True в случае успеха, False - иначе.
        """
        
        trace('[CHAP Server]', f'''Attempt to login with: {login = }, 
        response = {bytes_as_hex(response)}''')
        
        if login not in self._db:
            trace('[CHAP Server]', "User doesn't exist")
            return False
        
        #
        # Пользователь есть, достану челлендж и обнулю
        #
        
        challenge = self._db[login][1]
        self._db[login][1] = None
        
        if challenge is None:
            trace('[CHAP Server]', f'No challenge was generated for {login = }')
            return False
        
        #
        # Рассчитаю хэш и сравню с респонсом
        #
        
        challenge_hash = streebog_hash(challenge, self._db[login][0])
        
        trace('[CHAP Server]', f'Calculated hash: {bytes_as_hex(challenge_hash)}')
        trace('[CHAP Server]', f'Received hash:   {bytes_as_hex(response)}')
        
        return challenge_hash == response
    
    
    def generate_challenge(self, login):
        """
        Генерация челленджа. На вход получает логин.
        Возвращает случайный челлендж в случае успеха, None - иначе.
        """
        
        trace('[CHAP Server]', f'Attempt to generate challege for: {login = }')
        
        if login not in self._db:
            trace('[CHAP Server]', "User doesn't exist")
            return None
        
        #
        # Если юзер есть в базе, то генерирую челлендж и сохраняю у себя
        #
        
        challenge = get_random_bytes(16)
        self._db[login][1] = challenge
        
        trace('[CHAP Server]', f'Generated challenge = {bytes_as_hex(challenge)}')
        return challenge

In [13]:
class CHAPClient(object):
    def __init__(self, login, password):
        """
        Инициализация клиента. На вход получает логин и пароль.
        """
        
        self._login  = login
        self._passwd = password
        
        trace('[CHAP Client]', f'User with credentials {self._login}:{self._passwd} created')
        
        
    def register(self, srv: CHAPServer):
        """
        Регистрация пользователя на сервере. На вход принимает дескриптор сервера.
        """
        
        try:
            srv.register_user(self._login, self._passwd)
        except Exception as e:
            trace(e)
    
    
    def login(self, srv: CHAPServer):
        """
        Вход пользователя на сервере. На вход принимает дескриптор сервера.
        """
        
        #
        # Получаю челлендж с сервера
        #
        
        challenge = srv.generate_challenge(self._login)
        if challenge is None:
            print('[CHAP Client]', f'Wrong username or password')
            return
        
        #
        # Хэширую челлендж с паролем и пытаюсь залогиниться
        #
        
        response = streebog_hash(challenge, self._passwd.encode())
        if srv.login(self._login, response):
            print('[CHAP Client]', f'User {self._login} logged in successfully')
        else:
            print('[CHAP Client]', f'Wrong username or password')

----

In [14]:
# Создаю сервер
server = CHAPServer()

In [15]:
# Теперь создаю двух юзеров: 
# - Алису - валидного пользователя
# - Еву - пользователя, пытающегося представиться Алисой
alice = CHAPClient('Alice', 'Trust_m3_0r_n0t')
eve = CHAPClient('Alice', 'try_t0_gu3$$')

[TRACE] [CHAP Client] User with credentials Alice:Trust_m3_0r_n0t created
[TRACE] [CHAP Client] User with credentials Alice:try_t0_gu3$$ created


In [16]:
# Регистрирую валидного пользователя
alice.register(server)

[TRACE] [CHAP Server] Attempt to register user with credentials Alice:Trust_m3_0r_n0t
[TRACE] [CHAP Server] User successfully registered


In [17]:
# Захожу под кредами валидного пользователя
alice.login(server)

[TRACE] [CHAP Server] Attempt to generate challege for: login = 'Alice'
[TRACE] [CHAP Server] Generated challenge = df2f5e99ecb56d196ff304cd737346a5
[TRACE] [CHAP Server] Attempt to login with: login = 'Alice', 
        response = f7fed27f536473bffc7d1eb00c2d4e3ac013e3f266c648e52bb9baa68eba93dc
[TRACE] [CHAP Server] Calculated hash: f7fed27f536473bffc7d1eb00c2d4e3ac013e3f266c648e52bb9baa68eba93dc
[TRACE] [CHAP Server] Received hash:   f7fed27f536473bffc7d1eb00c2d4e3ac013e3f266c648e52bb9baa68eba93dc
[CHAP Client] User Alice logged in successfully


In [18]:
# Пытаюсь зайти под неправильными кредами
eve.login(server)

[TRACE] [CHAP Server] Attempt to generate challege for: login = 'Alice'
[TRACE] [CHAP Server] Generated challenge = 05755a626406f82da9f646a6f75cae21
[TRACE] [CHAP Server] Attempt to login with: login = 'Alice', 
        response = c1da97f11a7e4a505d67267d66ea5f10c430601ae822d883e49bc5290e5cb112
[TRACE] [CHAP Server] Calculated hash: caba03a5b79793394ba4edd0d86b104e659ce314904e1a8feb65ed5ae21f81cd
[TRACE] [CHAP Server] Received hash:   c1da97f11a7e4a505d67267d66ea5f10c430601ae822d883e49bc5290e5cb112
[CHAP Client] Wrong username or password


----

## Двустронний протокол CHAP

In [19]:
class ModifiedCHAPServer(object):
    def __init__(self):
        """
        Инициализация сервера. 
        Создается пустая база с пользователями. Задаются логин и пароль.
        """
        
        self._db = {}
        self._server_login = 'Alice'
        self._server_pass  = '1qazXSW@'
    
    
    def register_user(self, login, password):
        """
        Регистрация юзера. На вход получает логин и пароль.
        Возвращает логин и пароль сервера (для регистрации на клиенте).
        Выбрасывает исключение в случае наличия пользователя в базе
        """
        
        trace('[Modified CHAP Server]', f'Attempt to register user with credentials {login}:{password}')
        
        if login in self._db:
            trace('[Modified CHAP Server]', 'User already exists')
            raise ValueError(f'User {login} is already registered')
            
        #
        # Запишу в базу
        #
        
        self._db[login] = [password.encode(), None]
        trace('[Modified CHAP Server]', f'User successfully registered')
        
        return self._server_login, self._server_pass
        
        
    def login(self, login, response, client_challenge):
        """
        Вход пользователя. На вход получает логин, хэш пароля с челленджем и челлендж клиента.
        Возвращает логин сервера и хэш клиентского челледжа в случае успеха, None - иначе.
        """
        
        trace('[Modified CHAP Server]', f'''Attempt to login with: {login = },
        response = {bytes_as_hex(response)}, 
        client_challenge = {bytes_as_hex(client_challenge)}''')
        
        if login not in self._db:
            trace('[Modified CHAP Server]', "User doesn't exist")
            return None
        
        #
        # Пользователь есть, достану челлендж и обнулю
        #
        
        challenge = self._db[login][1]
        self._db[login][1] = None
        
        if challenge is None:
            trace('[Modified CHAP Server]', f'No challenge was generated for {login = }')
            return None
        
        #
        # Рассчитаю хэш и сравню с респонсом
        #
        
        challenge_hash = streebog_hash(challenge, self._db[login][0])
        
        trace('[Modified CHAP Server]', f'Calculated hash: {bytes_as_hex(challenge_hash)}')
        trace('[Modified CHAP Server]', f'Received hash:   {bytes_as_hex(response)}')
        
        if challenge_hash != response:
            return None
        
        #
        # Теперь считаю хэш для челленджа клиента и возвращаю его
        #
        
        client_challenge_hash = streebog_hash(client_challenge, self._server_pass.encode())
        return self._server_login, client_challenge_hash
    
    
    def generate_challenge(self, login):
        """
        Генерация челленджа. На вход получает логин.
        Возвращает случайный челлендж в случае успеха, None - иначе.
        """
        
        trace('[Modified CHAP Server]', f'Attempt to generate challege for: {login = }')
        
        if login not in self._db:
            trace('[Modified CHAP Server]', "User doesn't exist")
            return None
        
        #
        # Если юзер есть в базе, то генерирую челлендж и сохраняю у себя
        #
        
        challenge = get_random_bytes(16)
        self._db[login][1] = challenge
        
        trace('[Modified CHAP Server]', f'Generated challenge = {bytes_as_hex(challenge)}')
        return challenge

In [20]:
class ModifiedCHAPClient(object):
    def __init__(self, login, password):
        """
        Инициализация клиента. На вход получает логин и пароль.
        Инициализируется пустая база серверов.
        """
        
        self._login  = login
        self._passwd = password
        
        self._servers_db = {}
        
        trace('[Modified CHAP Client]', f'User with credentials {self._login}:{self._passwd} created')
        
        
    def register(self, srv: ModifiedCHAPServer):
        """
        Регистрация пользователя на сервере. На вход принимает дескриптор сервера.
        """
        
        try:
            login, password = srv.register_user(self._login, self._passwd)
            self._servers_db[login] = password
        except Exception as e:
            trace(e)
    
    
    def login(self, srv: ModifiedCHAPServer):
        """
        Вход пользователя на сервере. На вход принимает дескриптор сервера.
        """
        
        #
        # Получаю челлендж с сервера
        #
        
        srv_challenge = srv.generate_challenge(self._login)
        if srv_challenge is None:
            print('[Modified CHAP Client]', f'Wrong username or password')
            return
        
        #
        # Хэширую челлендж с паролем, генерирую свой и пытаюсь залогиниться
        #
        
        response  = streebog_hash(srv_challenge, self._passwd.encode())
        challenge = self.generate_challenge()
        srv_data  = srv.login(self._login, response, challenge)
        
        if srv_data is not None:
            srv_login, srv_response = srv_data
            
            if srv_login not in self._servers_db:
                print('[Modified CHAP Client]', f'Server {srv_login} not registered')
                return
            
            #
            # Сервер есть в базе, сверяю хэши
            #
            
            srv_challenge_hash = streebog_hash(challenge, self._servers_db[srv_login].encode())
            
            trace('[Modified CHAP Client]', f'Calculated hash: {bytes_as_hex(srv_challenge_hash)}')
            trace('[Modified CHAP Client]', f'Received hash:   {bytes_as_hex(srv_response)}')
            
            if srv_challenge_hash == srv_response:
                print('[Modified CHAP Client]', f'User {self._login} logged in successfully')
            else:
                print('[Modified CHAP Client]', f'Server {srv_login} is not authenticated')
        
        print('[Modified CHAP Client]', f'Wrong username or password')
        
        
    @staticmethod
    def generate_challenge():
        """
        Генерация челленджа.
        Возвращает случайное число.
        """
        
        return get_random_bytes(16)

------

In [21]:
# Создаю сервер
server = ModifiedCHAPServer()

In [22]:
# Теперь создаю двух юзеров: 
# - Алису - валидного пользователя
# - Еву - пользователя, пытающегося представиться Алисой
alice = ModifiedCHAPClient('Alice', 'Trust_m3_0r_n0t')
eve = ModifiedCHAPClient('Alice', 'try_t0_gu3$$')

[TRACE] [Modified CHAP Client] User with credentials Alice:Trust_m3_0r_n0t created
[TRACE] [Modified CHAP Client] User with credentials Alice:try_t0_gu3$$ created


In [23]:
# Регистрирую валидного пользователя
alice.register(server)

[TRACE] [Modified CHAP Server] Attempt to register user with credentials Alice:Trust_m3_0r_n0t
[TRACE] [Modified CHAP Server] User successfully registered


In [24]:
# Захожу под кредами валидного пользователя
alice.login(server)

[TRACE] [Modified CHAP Server] Attempt to generate challege for: login = 'Alice'
[TRACE] [Modified CHAP Server] Generated challenge = fbbb6d6d2a39b5b4efcb382cc021ef9a
[TRACE] [Modified CHAP Server] Attempt to login with: login = 'Alice',
        response = 635e80e68e54322ef9db6cf4dbfc165929d8f6528cb850bbc4c7db31d9adfb55, 
        client_challenge = fe38978f80bdc67f3840343366282335
[TRACE] [Modified CHAP Server] Calculated hash: 635e80e68e54322ef9db6cf4dbfc165929d8f6528cb850bbc4c7db31d9adfb55
[TRACE] [Modified CHAP Server] Received hash:   635e80e68e54322ef9db6cf4dbfc165929d8f6528cb850bbc4c7db31d9adfb55
[TRACE] [Modified CHAP Client] Calculated hash: ddd831a9b956dab4caca94ced2951e0011c9e7a5d662f687d59cac0e62cdf8a7
[TRACE] [Modified CHAP Client] Received hash:   ddd831a9b956dab4caca94ced2951e0011c9e7a5d662f687d59cac0e62cdf8a7
[Modified CHAP Client] User Alice logged in successfully
[Modified CHAP Client] Wrong username or password


In [25]:
# Пытаюсь зайти под неправильными кредами
eve.login(server)

[TRACE] [Modified CHAP Server] Attempt to generate challege for: login = 'Alice'
[TRACE] [Modified CHAP Server] Generated challenge = 10a98533a66e49d601bb6d34fb1f7caa
[TRACE] [Modified CHAP Server] Attempt to login with: login = 'Alice',
        response = 1980f25b39affe1b4c22449ab44142b331ba7fe10b246093facb35d5db572cc3, 
        client_challenge = 829275511b94caaf1d9b86569a3767c2
[TRACE] [Modified CHAP Server] Calculated hash: e32405318cc0f2914ada3b3ab448da61176a54bfc5542018b9e0113e11cf9647
[TRACE] [Modified CHAP Server] Received hash:   1980f25b39affe1b4c22449ab44142b331ba7fe10b246093facb35d5db572cc3
[Modified CHAP Client] Wrong username or password


----

## Протокол S/KEY
![S/KEY Protocol](./images/SKEY_1.png)

In [26]:
def generate_passwords(key, salt, number_of_passwords):
    """
    Генерирует последовательность паролей по ключу и соли в заданном количестве штук.
    Возвращает список паролей.
    """
    
    result = []
    intermediate = key + salt
    
    for _ in range(SKEYClient.ROUNDS):
        intermediate = streebog_hash(intermediate)
        result.append(intermediate)
        
    return result

----

In [27]:
class SKEYServer(object):
    def __init__(self, rounds):
        """
        Инициализация сервера. 
        Создается пустая база с пользователями. Задается количество итераций.
        """
        
        self._db = {}  # { login: [key, transaction_number, salt, passwords, current_password]}
        self._rounds = rounds
        
        
    def register_user(self, login, key):
        """
        Регистрация юзера. На вход получает логин и ключ.
        Возвращает соль для этого клиента.
        Выбрасывает исключение в случае наличия пользователя в базе
        """
        
        trace('[S/KEY Server]', f'Attempt to register user {login} with key {bytes_as_hex(key)}')
        
        if login in self._db:
            trace('[S/KEY Server]', 'User already exists')
            raise ValueError(f'User {login} is already registered')
            
        self._db[login] = [key, 1, None, [], None]
        salt = self.update_salt(login)
        
        trace('[S/KEY Server]', f'User successfully registered')
        return salt
    
    
    def login(self, login, nth_password):
        """
        Вход пользователя. На вход получает логин и сессионный пароль.
        Возвращает 1 в случае успеха, 0 в случае необходимости в 
        следующий раз обновить соль, -1 - иначе.
        """
        
        trace('[S/KEY Server]', f'''Attempt to login with: {login = },
        nth_password = {bytes_as_hex(nth_password)}''')
        
        if login not in self._db:
            trace('[S/KEY Server]', "User doesn't exist")
            return -1
        
        #
        # Если пользователь есть, а пароль верный, то инкременирую счетчик
        # и успешно возвращаю управление. Иначе - просто отдаю управление.
        #
        
        if nth_password != self._db[login][4]:
            return -1
        
        new_transaction_number = self._increment_transaction_number(login)
        if -1 == new_transaction_number:
            return 0
        
        self._db[login][4] = self._db[login][3][-new_transaction_number]
        return 1
    
    
    def get_transaction_number(self, login):
        """
        Возвращает номер транзации для заданного пользователя 
        или None, если такого пользователя нет.
        """
        
        trace('[S/KEY Server]', f'Attempt to get transaction number for: {login = }')
        
        if login not in self._db:
            trace('[S/KEY Server]', "User doesn't exist")
            return None
              
        return self._db[login][1]
    
    
    def update_salt(self, login):
        """
        Обновление соли для пользователя. Проверки существования пользователя
        не производятся, так как предполагается, что метод будет вызываться
        только для уже существующего пользователя.
        """
        
        trace('[S/KEY Server]', f'Attempt to generate salt for {login = }')
        
        self._generate_salt_and_passwords(login)
        self._db[login][1] = 1
        
        trace('[S/KEY Server]', f'Salt for user {login} successfully updated')
        return self._db[login][2]
    
    
    def _generate_salt_and_passwords(self, login):
        """
        Генерирует соль для пользователя и последовательность паролей.
        Проверки существования пользователя не производятся, так как 
        предполагается, что метод будет вызываться только для уже 
        существующего пользователя.
        На вход принимает логин.
        Возвращает соль.
        """
        
        salt = get_random_bytes(16)
        self._db[login][2] = salt
        
        trace('[S/KEY Server]', f'Generated salt = {bytes_as_hex(salt)}')
        self._db[login][3] = generate_passwords(self._db[login][0], self._db[login][2], self._rounds)
        self._db[login][4] = self._db[login][3][-1]
        
        trace('[S/KEY Server]', 'Passords generated')
        return salt
    
    
    def _increment_transaction_number(self, login):
        """
        Инкрементирует счетчик для заданного пользователя.
        Возвращает новое значение счетчика или -1, если необходима смена соли.
        """
        
        self._db[login][1] += 1
        
        if self._db[login][1] > self._rounds:
            trace('[S/KEY Server]', 'Passwords number exceeded')
            return -1
            
        return self._db[login][1]

In [28]:
class SKEYClient(object):
    ROUNDS = 2
    
    def __init__(self, login, key):
        """
        Инициализация клиента. На вход получает логин и ключ.
        Инициализируется пустые соль и список паролей.
        """
        
        self._login = login
        self._key = key
        self._salt = None
        self._passwords = []
        
        trace('[S/KEY Client]', f'User {self._login} created, key = {bytes_as_hex(key)}')
        
        
    def register(self, server: SKEYServer):
        """
        Регистрация пользователя на сервере. На вход принимает дескриптор сервера.
        """
        
        try:
            salt = server.register_user(self._login, self._key)
            self._passwords = generate_passwords(self._key, salt, SKEYClient.ROUNDS)
            
            trace('[S/KEY Client]', 'Passwords generated')
        except Exception as e:
            trace(e)
    
    
    def login(self, server: SKEYServer):
        """
        Вход пользователя на сервере. На вход принимает дескриптор сервера.
        """
        
        transaction_number = server.get_transaction_number(self._login)
        if transaction_number is None:
            print('[S/KEY Client]', f'Wrong username or password')
            return
            
        trace('[S/KEY Client]', f'Transaction number {transaction_number}')
        
        login_result = server.login(self._login, self._passwords[-transaction_number])
        if login_result == 1:
            print('[S/KEY Client]', f'User {self._login} logged in successfully')
            
        elif login_result == 0:
            print('[S/KEY Client]', f'User {self._login} logged in successfully')
            
            new_salt = server.update_salt(self._login)
            self._passwords = generate_passwords(self._key, new_salt, SKEYClient.ROUNDS)
            trace('[S/KEY Client]', f'Salt updated successfully')
        else:
            print('[S/KEY Client]', f'Wrong username or password')
    

----

In [29]:
# Создаю сервер
server = SKEYServer(SKEYClient.ROUNDS)

In [30]:
# Создаю случайный ключ (ну можно и фиксированный для 
# демонстрации, но пусть будет случайным)
key = get_random_bytes(16)
bytes_as_hex(key)

'831cb542314bb7d46afc6d44c458caa5'

In [31]:
# Теперь создаю юзера:
alice = SKEYClient('Alice', key)

[TRACE] [S/KEY Client] User Alice created, key = 831cb542314bb7d46afc6d44c458caa5


In [32]:
# Регистрирую пользователя
alice.register(server)

[TRACE] [S/KEY Server] Attempt to register user Alice with key 831cb542314bb7d46afc6d44c458caa5
[TRACE] [S/KEY Server] Attempt to generate salt for login = 'Alice'
[TRACE] [S/KEY Server] Generated salt = 64adf48ddccea67e09fa977014b22a7c
[TRACE] [S/KEY Server] Passords generated
[TRACE] [S/KEY Server] Salt for user Alice successfully updated
[TRACE] [S/KEY Server] User successfully registered
[TRACE] [S/KEY Client] Passwords generated


In [33]:
# Захожу под кредами пользователя
alice.login(server)

[TRACE] [S/KEY Server] Attempt to get transaction number for: login = 'Alice'
[TRACE] [S/KEY Client] Transaction number 1
[TRACE] [S/KEY Server] Attempt to login with: login = 'Alice',
        nth_password = 0d45b66ee85146d2374912f2708ff87069c64594cd2c7bafd2edbed675673114
[S/KEY Client] User Alice logged in successfully


In [34]:
# И еще раз, тут должна произойти последняя попытка
# аутентификации и, как следствие, обновление соли автоматическое
alice.login(server)

[TRACE] [S/KEY Server] Attempt to get transaction number for: login = 'Alice'
[TRACE] [S/KEY Client] Transaction number 2
[TRACE] [S/KEY Server] Attempt to login with: login = 'Alice',
        nth_password = aaa6ed5717cf870b6cf54626703441aa51a16934548180a20ac3399429f28931
[TRACE] [S/KEY Server] Passwords number exceeded
[S/KEY Client] User Alice logged in successfully
[TRACE] [S/KEY Server] Attempt to generate salt for login = 'Alice'
[TRACE] [S/KEY Server] Generated salt = a95f0ddee68e8f565cd20f61a60f648c
[TRACE] [S/KEY Server] Passords generated
[TRACE] [S/KEY Server] Salt for user Alice successfully updated
[TRACE] [S/KEY Client] Salt updated successfully


In [35]:
# И еще раз логинимся. Соль должна быть уже другой.
alice.login(server)

[TRACE] [S/KEY Server] Attempt to get transaction number for: login = 'Alice'
[TRACE] [S/KEY Client] Transaction number 1
[TRACE] [S/KEY Server] Attempt to login with: login = 'Alice',
        nth_password = 384b813bd54ed848e2ba45ebbe100d3350b8c8f2590181b6abf3faa4f38513d5
[S/KEY Client] User Alice logged in successfully
